# Motivation

论文原文：[《Deep & Cross Network for Ad Click Predictions》](https://arxiv.org/abs/1708.05123)

参考学习地址：[揭秘 Deep & Cross : 如何自动构造高阶交叉特征](https://zhuanlan.zhihu.com/p/55234968)

针对大规模稀疏特征的点击率预估问题，该篇论文是Google对Wide&Deep（同时实现了Memorization与Generalization）工作的一个后续研究，但在Wide部分仍需人工设计特征叉乘，FM可以自动组合，但也仅限于二阶叉乘。文中提出Deep&Cross Network，受 **<font color="red">残差网络（Residual Network，参考ResNet，注意不是RNN）启发</font>**，旨在用特殊网络结构实现的Cross替换Wide部分，从而实现**显式的自动构造有效高阶的交叉特征**，并学习对应的权重，告别了繁琐的人工叉乘。

 <img style="display: block; margin: 0 auto;" src="../../../assets/images/dcn.png" width = "500" height = "600" alt="DCN网络结构" align=center />

## Embedding and stacking layer

原始特征做如下处理：
* sparse特征进行embedding，对于 multi-hot 的sparse特征，embedding之后再average pooling
* dense特征归一化，然后和embedding特征拼接

拼接特征特征作为随后Cross层与Deep层的共同输入。

$$
\begin{align}
x_0 = [x_{embed,1}^T, x_{embed,2}^T, ..., x_{embed,k}^T, x_{dense}^T]
\end{align}
$$

其中 $x_{embed,i}^T=W_{embed,i}x_i$ 表示embedding向量，$x_i$代表第$i$个类别对应的binary输入，$W_{embed,i}\in R^{n_e\times n_v}$ 表示对应的embedding矩阵（和其他参数一样会在训练时被优化），并且$n_e$和$n_v$分别代表了embedding size 和vocabulary size。

## Cross Layer

Cross的目的是以一种 **显式、可控且高效** 的方式，自动构造有限高阶交叉特征。

 <img style="display: block; margin: 0 auto;" src="../../../assets/images/dcn-cross.png" width = "400" height = "200" alt="DCN-Cross网络结构" align=center />

 其核心思想是：通过引入残差网络（Residual Network，参考ResNet而非RNN）结构，使每一层的输出包含上一层的输出，每层实际拟合的是层间的残差（<font color="red">残差网络结构之中一个优点是处理梯度消失，使网络可以“更深”</font>）

 $$
\begin{align}
x_{l+1}=x_0x_l^Tw_l+b_l+x_l=f(x_l,w_l,b_l)+x_l
\end{align}
$$

其中$x_l,x_{l+1}\in R^d$是第$l$和$l+1$层的输出，$w_l,b_l$代表第$l$层的权重和偏置。每一个cross层在特征交叉函数$f$（$f:R^d \rightarrow R^d$拟合了$x_{l+1}-x_l$的残差）后，再和该层的输入相加

Tips：注意前提条件是每层的输入输出维度都是相等的。

假设此时cross有两层，embedding后$x_0=\begin{bmatrix}{x_{0,1}}\\{x_{0,2}}\\\end{bmatrix}$，偏置恒为0，则有:

$$
\begin{align}
x_1 &= x_0x_0^Tw_0 + x_0\\
&=\begin{bmatrix}{x_{0,1}}\\{x_{0,2}}\\\end{bmatrix}[x_{0,1},x_{0,2}]\begin{bmatrix}{w_{0,1}}\\{w_{0,2}}\\\end{bmatrix}+\begin{bmatrix}{x_{0,1}}\\{x_{0,2}}\\\end{bmatrix}\\
&=\begin{bmatrix}{w_{0,1}x_{0,1}^2}+w_{0,2}x_{0,1}x_{0,2}+x_{0,1}\\{w_{0,1}x_{0,2}x_{0,1}}+w_{0,2}x_{0,2}^2+x_{0,2}\\\end{bmatrix}\\
x_2 &= x_0x_1^Tw_1 + x_1\\
&=\begin{bmatrix}{x_{0,1}}\\{x_{0,2}}\\\end{bmatrix}[x_{1,1},x_{1,2}]\begin{bmatrix}{w_{1,1}}\\{w_{1,2}}\\\end{bmatrix}+\begin{bmatrix}{x_{1,1}}\\{x_{1,2}}\\\end{bmatrix}\\
&=\begin{bmatrix}{x_{0,1}}\\{x_{0,2}}\\\end{bmatrix}[{w_{0,1}x_{0,1}^2}+w_{0,2}x_{0,1}x_{0,2}+x_{0,1},{w_{0,1}x_{0,2}x_{0,1}}+w_{0,2}x_{0,2}^2+x_{0,2}]\begin{bmatrix}{w_{1,1}}\\{w_{1,2}}\\\end{bmatrix}+\begin{bmatrix}{{w_{0,1}x_{0,1}^2}+w_{0,2}x_{0,1}x_{0,2}+x_{0,1}}\\{{w_{0,1}x_{0,2}x_{0,1}}+w_{0,2}x_{0,2}^2+x_{0,2}}\\\end{bmatrix}\\
&=\begin{bmatrix}{x_{0,1}w_{1,1}x_{0,1}^3 + (w_{0,2}w_{1,1}+w_{0,1}w_{1,2})x_{0,1}^2x_{0,2} + w_{0,2}w_{1,2}x_{0,1}x_{0,2}^2+(w_{0,1}+w_{1,1})x_{0,1}^2+(w_{0,2}+w_{1,2})x_{0,1}x_{0,2}+x_{0,1}}\\{...}\\\end{bmatrix}\\
\end{align}
$$

可以看到第$l$层的输出中，每个向量输出的位置都包含了该位置对应$x_{0,i}$向量的所有可能叉乘组合。Cross的设计总结如下：
* 有限高阶：叉乘阶数由网络深度决定，深度$L_c$对应最高$L_c+1$阶的叉乘
* 自动叉乘：Cross输出包含了原始特征从一阶（i.e.本身）到$L_c+1$阶的所有叉乘组合，且模型的参数量仅随维度成线性增长：$2*d*L_c$
* 参数共享：不同叉乘项对应的权重不同，但并非每个叉乘组合对应独立的权重（指数数量级），通过参数共享，Cross有效降低了参数量。此外，参数共享还使得模型有更强的泛化性和鲁棒性。e.g. 如果独立训练参数，当训练集中没有$x_i0x_j0$这个叉乘特征没有出现，对应权重肯定是零，但参数共享不会。类似地，数据集中的一些噪声可以有大部分正常样本来纠正参数学习。

## Deep & Combined output
Deep层为传统FFN结构，Cross和Deep输出stack后原文中也<font color="red">在目标函数中加入了L2正则防止过拟合</font>。

## 一些思考
1. 文中将dense特征和embedding特征拼接后作为Cross层和Deep层的共同输入。这对于Deep层是合理的，但人工交叉特征基本是对原始sparse特征进行叉乘，考虑到Cross设计中 **<font color="red">每层layer的节点数都与Cross的输入维度一致的</font>**，所以直接使用大规模高维的sparse特征作为输入，会导致极大地增加Cross的参数量，实则是trade-off的选择（理论上sparse部分应不embedding直接喂给Cross层）。

2. 模型参数量分析，假设初始输入$x_0$维度为$d$，Deep和Cross层数分别为$L_{cross}$和$L_{deep}$，为便于分析，设Deep每层神经元个数为$m$，则两部分参数分别为：
   * $\#Cross=d*L_{cross}*2$
   * $\#Deep=\underbrace{(d*m+m)}_{第一层接受Embedding结果，加号代表偏置}+\underbrace{(m^2+m)*{L_{deep}-1}}_{中间层节点wx+b}$

3. （xDeepFM中证明，其实也是ResNet结构+Cross参数共享导致的叉乘学习模式固定）Cross部分的高阶叉乘退化为$x_0$的常数倍乘积。例如$x_1$的推理公式进一步提取公项：
    $$
    \begin{align}
    x_1 &=\begin{bmatrix}{w_{0,1}x_{0,1}^2}+w_{0,2}x_{0,1}x_{0,2}+x_{0,1}\\{w_{0,1}x_{0,2}x_{0,1}}+w_{0,2}x_{0,2}^2+x_{0,2}\\\end{bmatrix}\\
    &=\begin{bmatrix}{w_{0,1}x_{0,1}+w_{0,2}x_{0,2}+1}\\{w_{0,1}x_{0,1}+w_{0,2}x_{0,2}+1}\end{bmatrix}[x_{0,1}, x_{0,2}]\\
    &=x_0(x_0^Tw_0+1)\\
    &=\alpha^1*x_0\\
    \end{align}
    $$
    其中$\alpha^1=w_{0,1}x_{0,1}+w_{0,2}x_{0,2}+1=x_0^Tw_0+1$实际上是$x_0$的线性回归。同理高阶可以得到$\alpha^{l+1}=\alpha^i(x_0^Tw_{i+1}+1)$，始终为$x_0$的常数倍乘积。虽然Cross可以高效的学习高阶特征交叉信息，但缺点是每层都被限制为了一个特殊形式（<font color="red">输入和输入的线性变换的乘积</font>），也即 a scalar multiple of $x_0$，尽管这不意味着$x_l$和$x_0$有线性关系。那考虑收拾在$x_0^Tw_{i+1}+1$部分加个激活函数即可呢？

4. 从Wide&Deep架构本身的motivation出发，deep部分目标是Generalization（复杂组合衍生），Wide部分目标是Memorization（记忆泛化性），如果不考虑DNN部分的组合是隐式（Implicitly），Wide部分不应该像Cross这样（Explicitly）做特征交叉工作。